In [1]:
import numpy as np
import sympy as sp
import math as mt
import pandas as pd

In [2]:
SIZE = 7

In [3]:
def AB_matrix(A, B):
    AB = A.col_insert(SIZE, B)
    return AB


def QR_decomposition(A, B):
    A = sp.Matrix(A)
    B = sp.Matrix(B)
    M = AB_matrix(A, B)
    
    df = {}
    
    for i in range(0, SIZE-1):
        
        df[i] = pd.DataFrame(np.array(M))
        
        p = np.zeros(SIZE, dtype=np.float64)
        p[i] = M.row(i)[i] + np.sign(M.row(i)[i]) * mt.sqrt(sum([(M.col(i)[k])**2 for k in range(i, SIZE)]))
        p[i+1:] = M.col(i)[i+1:]
        
        df[i] = pd.concat([df[i], pd.DataFrame(p)], axis = 1)
        df[i] = pd.concat([df[i], pd.DataFrame(np.full((3, i), np.nan))], axis = 1)
        
        s = sum([p[j]**2 for j in range(SIZE)])
        r = np.array([sum([p[j] * M.col(i+k+1)[j] for j in range(SIZE)]) for k in range(SIZE-i)], dtype=np.float64)
        z = np.array([[(2 * r[j] * p[k] / s) for j in range(0, len(r))] for k in range(SIZE)], dtype=np.float64)
        
        BLOCK = sp.Matrix(z)
        BLOCK = BLOCK.row_insert(SIZE, sp.Matrix(r.reshape(1, SIZE-i)))
        df[i] = pd.concat([df[i], pd.DataFrame(np.array(BLOCK))], axis = 1, ignore_index = True)
        df[i].loc[df[i].index == SIZE, SIZE+1] = s  
        
        new_M = np.array(M, dtype=np.float64)
        new_M[i][i] -= p[i]
        for k in range(i+1, SIZE):
            new_M[k][i] = 0
        for k in range(i, SIZE):
            for j in range(i+1, SIZE+1):
                new_M[k][j] -= z[k][j-i-1]
        M = sp.Matrix(new_M)
        
        if (i==0):
            result_table = df[0]
        else:
            result_table = pd.concat([result_table, df[i]], axis = 0, ignore_index=False)
    
    return M, result_table
        

def reflection_method(A, B):
    M, table = QR_decomposition(A, B)
    
    table = table.astype(np.float64).round(4)
    
    last_colums = np.array([f'a_i{j+1}' if j < SIZE+1 else 'x' for j in range(SIZE+2)]).reshape(1, SIZE+2)
    table = pd.concat([table, pd.DataFrame(last_colums)], axis = 0, ignore_index=False)
    
    x = np.zeros(SIZE, dtype=np.float64)
    x[SIZE-1] = M.row(SIZE-1)[SIZE] / M.row(SIZE-1)[SIZE-1]
    for reversed_index in range(SIZE-2, -1, -1):
        x[reversed_index] = (M.col(SIZE)[reversed_index] - sum([M.row(reversed_index)[j] * x[j] for j in range(reversed_index+1, SIZE)])) / M.row(reversed_index)[reversed_index]
    
    last_part = np.array([[np.array(M)[i][j] if j <= SIZE else x[i] for j in range(SIZE+2)] for i in range(SIZE)], dtype=np.float64)
    
    last_part = last_part.round(4)
    
    table = pd.concat([table, pd.DataFrame(last_part)], axis = 0, ignore_index=False)
    
    table.columns = [f'A{j+1}' if j<(SIZE+1) else 'p' for j in range(2*(SIZE+1))]
    table.columns = [table.columns[j] if j<(SIZE+2) else f'z{j-(SIZE)}' for j in range(2*(SIZE+1))]
    
    return sp.Matrix(x), table

In [4]:
A = np.array([
    [2.2, 4, -3, 1.5, 0.6, 2, 0.7],
    [4, 3.2, 1.5, -0.7, -0.8, 3, 1],
    [-3, 1.5, 1.8, 0.9, 3, 2, 2],
    [1.5, -0.7, 0.9, 2.2, 4, 3, 1],
    [0.6, -0.8, 3, 4, 3.2, 0.6, 0.7],
    [2, 3, 2, 3, 0.6, 2.2, 4],
    [0.7, 1, 2, 1, 0.7, 4, 3.2]
], dtype = np.float64)

B = np.array([3.2, 4.3, -0.1, 3.5, 5.3, 9.0, 3.7], dtype = np.float64)

In [5]:
if (np.linalg.det(A)!=0):
    x, TABLE = reflection_method(A, B)
print("Ответ:")
x

Ответ:


Matrix([
[  1.11843651252141],
[ 0.226402009944434],
[ 0.302457443252382],
[ 0.687367484631095],
[ 0.222326641793347],
[-0.617868162866287],
[  1.16070439448892]])

In [6]:
error = sp.Matrix(A) * x - sp.Matrix(B)
print("Погрешность:")
error

Погрешность:


Matrix([
[ 1.33226762955019e-15],
[                    0],
[-9.71445146547012e-16],
[ 1.77635683940025e-15],
[ 1.77635683940025e-15],
[ 1.77635683940025e-15],
[                    0]])

In [7]:
print("Максимум погрешности:", max(error))

Максимум погрешности: 1.77635683940025e-15


In [8]:
TABLE

,A1,A2,A3,A4,A5,A6,A7,A8,p,z2,z3,z4,z5,z6,z7,z8
0,2.2,4.0,-3.0,1.5,0.6,2.0,0.7,3.2,8.2778,7.6641,-2.5804,3.1782,0.3910,5.6954,2.6250,12.0124
1,4.0,3.2,1.5,-0.7,-0.8,3.0,1.0,4.3,4.0,3.7035,-1.2469,1.5358,0.1890,2.7521,1.2685,5.8046
2,-3.0,1.5,1.8,0.9,3.0,2.0,2.0,-0.1,-3.0,-2.7776,0.9352,-1.1518,-0.1417,-2.0641,-0.9513,-4.3534
3,1.5,-0.7,0.9,2.2,4.0,3.0,1.0,3.5,1.5,1.3888,-0.4676,0.5759,0.0709,1.0320,0.4757,2.1767
4,0.6,-0.8,3.0,4.0,3.2,0.6,0.7,5.3,0.6,0.5555,-0.1870,0.2304,0.0283,0.4128,0.1903,0.8707
5,2.0,3.0,2.0,3.0,0.6,2.2,4.0,9.0,2.0,1.8517,-0.6235,0.7679,0.0945,1.3761,0.6342,2.9023
6,0.7,1.0,2.0,1.0,0.7,4.0,3.2,3.7,0.7,0.6481,-0.2182,0.2688,0.0331,0.4816,0.2220,1.0158
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.6224,46.5813,-15.6835,19.3167,2.3767,34.6157,15.9545,73.0091
0,-6.0778,-3.6641,-0.4196,-1.6782,0.209,-3.6954,-1.925,-8.8124,0.0,NaN,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,0.0,-0.5035,2.7469,-2.2358,-0.989,0.2479,-0.2685,-1.5046,-5.6215,NaN,2.9555,-3.0605,-1.4277,-2.6985,-3.3725,-5.0470


In [9]:
# TABLE.to_excel('Reflection_Method_Steps.xlsx')